In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from pathlib import Path

from tqdm.notebook import tqdm
import plotly.graph_objects as go
import plotly.subplots as sp
import plotly.io as pio
import plotly.express as px
from itertools import cycle
pio.templates.default = "plotly_white"
import tensorflow as tf

import sys
import os
sys.path.append(os.path.abspath('../'))

from src.utils import plotting_utils
from src.dl.dataloaders import TimeSeriesDataModule
from src.dl.multivariate_models import SingleStepRNNConfig, SingleStepRNNModel


tqdm.pandas()

torch.set_float32_matmul_precision("high")
# pl.seed_everything(100)

seed = 100
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
    

c:\Users\olarinoyem\Project\EpiHealthForecast\src\utils\data_utils.py:6: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
source_data = Path("../data/")
# Check if a GPU is available
if tf.config.list_physical_devices("GPU"):
    print("GPU is available")
else:
    print("GPU is not available")

GPU is available


In [4]:
data = pd.read_csv(source_data / "processed" / "merged_nhs_covid_data.csv")
data.head()

,areaName,date,covidOccupiedMVBeds,cumAdmissions,hospitalCases,newAdmissions,new_confirmed,new_deceased,cumulative_confirmed,cumulative_deceased,population,openstreetmap_id,latitude,longitude
0,East of England,2020-04-01,0.0,1400,833.0,167,334.0,75.0,2938.0,455.0,6235410,151336,52.24,0.41
1,East of England,2020-04-02,119.0,1584,841.0,184,372.0,71.0,3310.0,526.0,6235410,151336,52.24,0.41
2,East of England,2020-04-03,162.0,1776,914.0,192,350.0,85.0,3660.0,611.0,6235410,151336,52.24,0.41
3,East of England,2020-04-04,171.0,1939,988.0,163,268.0,70.0,3928.0,681.0,6235410,151336,52.24,0.41
4,East of England,2020-04-05,219.0,2159,1230.0,220,281.0,91.0,4209.0,772.0,6235410,151336,52.24,0.41


In [5]:
# list all the unique values in the 'areaName' column
data['areaName'].unique()

array(['East of England', 'London', 'Midlands',
       'North East and Yorkshire', 'North West', 'South East',
       'South West'], dtype=object)

In [6]:
# Aggregate data to a single 'England' region
data = data.groupby('date').agg({
    'covidOccupiedMVBeds': 'sum',
    'cumAdmissions': 'sum',
    'hospitalCases': 'sum',
    'newAdmissions': 'sum',
    'new_confirmed': 'sum',
    'new_deceased': 'sum',
    'cumulative_confirmed': 'sum',
    'cumulative_deceased': 'sum',
    'population': 'sum',
    'openstreetmap_id': 'first',
    'latitude': 'first',
    'longitude': 'first'
}).reset_index()
data['areaName'] = 'England'

# Create timeseries features
data['date'] = pd.to_datetime(data['date'])
data['year'] = data['date'].dt.year
data['month'] = data['date'].dt.month
data['day'] = data['date'].dt.day
data['day_of_week'] = data['date'].dt.dayofweek

In [7]:
# Vax Index Calculation Function (As previously defined)
def calculate_vax_index(df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate the Vax index based on vaccination rates and efficacy across age groups.
    
    Args:
        df (pd.DataFrame): DataFrame containing 'date', 'cumAdmissions', 'cumulativeCases', and other relevant columns.
    
    Returns:
        pd.DataFrame: DataFrame with an additional 'Vax_index' column.
    """
    # Constants
    total_population = 60_000_000
    number_of_age_groups = 5
    vaccine_efficacy_first_dose = [0.89, 0.427, 0.76, 0.854, 0.75]  # Added 5th value
    vaccine_efficacy_second_dose = [0.92, 0.86, 0.81, 0.85, 0.80]  # Replaced None with 0.80
    age_group_probabilities_icu = [0.01, 0.02, 0.05, 0.1, 0.15]
    monthly_vaccination_rate_increase = 0.05
    vaccination_start_date = pd.Timestamp('2021-01-18')
    
    # Population per age group
    population_per_age_group = total_population / number_of_age_groups
    
    # Initialize Vax index list
    vax_index_list = []
    
    # Monthly vaccination rate (starting from 0)
    monthly_vaccination_rate = 0.0
    
    for index, row in df.iterrows():
        # Increment monthly vaccination rate on the first day of each month after start date
        if row['date'].day == 1 and row['date'] >= vaccination_start_date:
            monthly_vaccination_rate += monthly_vaccination_rate_increase
            # Ensure vaccination rate does not exceed 1
            monthly_vaccination_rate = min(monthly_vaccination_rate, 1.0)
            print(f"Updated monthly vaccination rate to {monthly_vaccination_rate} on {row['date']}")
        
        Si_sum = 0.0
        
        for i in range(number_of_age_groups):
            # Vaccinated population for this age group
            vaccinated_population = monthly_vaccination_rate * population_per_age_group
            
            # Assume half received first dose and half received second dose
            aij = vaccinated_population / 2  # First dose
            bij = vaccinated_population / 2  # Second dose
            cij = population_per_age_group - aij - bij  # Unvaccinated
            
            # Calculate S''i based on vaccine efficacy
            # Ensuring indices match
            S_double_prime_i = (vaccine_efficacy_second_dose[i] * bij +
                                 vaccine_efficacy_first_dose[i] * aij)
            
            # Calculate Si
            Si = aij + bij + cij - S_double_prime_i  # Effective susceptible
            
            # Age-specific probability
            pi = age_group_probabilities_icu[i]
            
            # Normalize Si by total population in age group
            Si_normalized = Si / population_per_age_group
            
            # Weighted sum
            Si_sum += pi * Si_normalized
        
        # Vax index for the day
        vax_index = Si_sum
        vax_index_list.append(vax_index)
    
    # Add Vax index to the dataframe
    df['Vax_index'] = vax_index_list
    print("Calculated Vax_index for all dates.")
    return df

# Calculate Vax index
data = calculate_vax_index(data)
data.head()

Updated monthly vaccination rate to 0.05 on 2021-02-01 00:00:00
Updated monthly vaccination rate to 0.1 on 2021-03-01 00:00:00
Updated monthly vaccination rate to 0.15000000000000002 on 2021-04-01 00:00:00
Updated monthly vaccination rate to 0.2 on 2021-05-01 00:00:00
Updated monthly vaccination rate to 0.25 on 2021-06-01 00:00:00
Updated monthly vaccination rate to 0.3 on 2021-07-01 00:00:00
Updated monthly vaccination rate to 0.35 on 2021-08-01 00:00:00
Updated monthly vaccination rate to 0.39999999999999997 on 2021-09-01 00:00:00
Updated monthly vaccination rate to 0.44999999999999996 on 2021-10-01 00:00:00
Updated monthly vaccination rate to 0.49999999999999994 on 2021-11-01 00:00:00
Updated monthly vaccination rate to 0.5499999999999999 on 2021-12-01 00:00:00
Updated monthly vaccination rate to 0.6 on 2022-01-01 00:00:00
Updated monthly vaccination rate to 0.65 on 2022-02-01 00:00:00
Updated monthly vaccination rate to 0.7000000000000001 on 2022-03-01 00:00:00
Updated monthly vacc

,date,covidOccupiedMVBeds,cumAdmissions,hospitalCases,newAdmissions,new_confirmed,new_deceased,cumulative_confirmed,cumulative_deceased,population,openstreetmap_id,latitude,longitude,areaName,year,month,day,day_of_week,Vax_index
0,2020-04-01,0.0,23332,12059.0,3099,3989.0,694.0,35571.0,4730.0,56171302,151336,52.24,0.41,England,2020,4,1,2,0.33
1,2020-04-02,1494.0,26264,12135.0,2932,3895.0,725.0,39466.0,5455.0,56171302,151336,52.24,0.41,England,2020,4,2,3,0.33
2,2020-04-03,1788.0,28828,13635.0,2564,3878.0,737.0,43344.0,6192.0,56171302,151336,52.24,0.41,England,2020,4,3,4,0.33
3,2020-04-04,1950.0,31421,15469.0,2593,3260.0,828.0,46604.0,7020.0,56171302,151336,52.24,0.41,England,2020,4,4,5,0.33
4,2020-04-05,2097.0,34013,16657.0,2592,2994.0,823.0,49598.0,7843.0,56171302,151336,52.24,0.41,England,2020,4,5,6,0.33


In [8]:
# Create a subplot with 6 rows (removing Vax_index as it's not in our data)
fig = sp.make_subplots(rows=5, cols=1, 
                       shared_xaxes=True, 
                       subplot_titles=(
                                     'New Hospital Admissions', 
                                     'Current Hospital Cases',
                                     'Mechanical Ventilator Bed Usage',
                                     'New COVID-19 Cases',
                                     'Vax Index'))

# Plot for New Hospital Admissions
fig.add_trace(go.Scatter(x=data['date'], 
                        y=data['newAdmissions'], 
                        line=dict(color='brown', width=2), 
                        name='New Admissions'), row=1, col=1)

# Plot for Current Hospital Cases
fig.add_trace(go.Scatter(x=data['date'], 
                        y=data['hospitalCases'], 
                        line=dict(color='green', width=2), 
                        name='Hospital Cases'), row=2, col=1)

# Plot for Mechanical Ventilators
fig.add_trace(go.Scatter(x=data['date'], 
                        y=data['covidOccupiedMVBeds'], 
                        line=dict(color='blue', width=2), 
                        name='Ventilator Beds'), row=3, col=1)

# Plot for New Cases
fig.add_trace(go.Scatter(x=data['date'], 
                        y=data['new_confirmed'], 
                        line=dict(color='orange', width=2), 
                        name='New Cases'), row=4, col=1)

# Plot for Vax Index
fig.add_trace(go.Scatter(x=data['date'], 
                        y=data['Vax_index'], 
                        line=dict(color='purple', width=2), 
                        name='Vax Index'), row=5, col=1)


# Update the layout
fig.update_layout(height=1200, 
                 width=800, 
                 title_text="COVID-19 Data visualisation for England",
                 showlegend=True)

fig.show()


c:\Users\olarinoyem\AppData\Local\miniconda3\envs\tf\lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [17]:
def format_plot(
    fig, legends=None, xlabel="Time", ylabel="Value", title="", font_size=15
):
    if legends:
        names = cycle(legends)
        fig.for_each_trace(lambda t: t.update(name=next(names)))
    fig.update_layout(
        autosize=False,
        width=900,
        height=500,
        title_text=title,
        title={"x": 0.5, "xanchor": "center", "yanchor": "top"},
        titlefont={"size": 20},
        legend_title=None,
        legend=dict(
            font=dict(size=font_size),
            orientation="h",
            yanchor="bottom",
            y=0.98,
            xanchor="right",
            x=1,
        ),
        yaxis=dict(
            title_text=ylabel,
            titlefont=dict(size=font_size),
            tickfont=dict(size=font_size),
        ),
        xaxis=dict(
            title_text=xlabel,
            titlefont=dict(size=font_size),
            tickfont=dict(size=font_size),
        ),
    )
    return fig


def mase(actual, predicted, insample_actual):
    mae_insample = np.mean(np.abs(np.diff(insample_actual)))
    mae_outsample = np.mean(np.abs(actual - predicted))
    return mae_outsample / mae_insample


def forecast_bias(actual, predicted):
    return np.mean(predicted - actual)


def plot_forecast(pred_df, forecast_columns, forecast_display_names=None, save_path=None):
    """
    Plot the forecasted values against actual values.
    
    Args:
        pred_df (pd.DataFrame): DataFrame containing actual and predicted values.
        forecast_columns (list): List of columns with forecasted data.
        forecast_display_names (list, optional): Display names for the forecasted columns.
        save_path (str, optional): Path to save the plot.
    
    Returns:
        plotly.graph_objects.Figure: The forecast plot.
    """
    if forecast_display_names is None:
        forecast_display_names = forecast_columns
    else:
        assert len(forecast_columns) == len(forecast_display_names)

    mask = ~pred_df[forecast_columns[0]].isnull()
    colors = px.colors.qualitative.Set2
    act_color = colors[0]
    colors = cycle(colors[1:])

    fig = go.Figure()

    # Actual Data Plot
    fig.add_trace(
        go.Scatter(
            x=pred_df[mask].index,
            y=pred_df['covidOccupiedMVBeds'][mask],  # Fixed column reference
            mode="lines",
            line=dict(color=act_color, width=2),
            name="Actual COVID-19 MVBeds Trends",
        )
    )

    # Predicted Data Plots
    for col, display_col in zip(forecast_columns, forecast_display_names):
        fig.add_trace(
            go.Scatter(
                x=pred_df[mask].index,
                y=pred_df.loc[mask, col],
                mode="lines+markers",
                marker=dict(size=4),
                line=dict(color=next(colors), width=2),
                name=display_col,
            )
        )

    return fig

def highlight_abs_min(s, props=""):
    return np.where(s == np.nanmin(np.abs(s.values)), props, "")

In [18]:
data

,date,covidOccupiedMVBeds,cumAdmissions,hospitalCases,newAdmissions,new_confirmed,new_deceased,cumulative_confirmed,cumulative_deceased,population,...,newAdmissions_rolling_mean_14,newAdmissions_rolling_std_14,Vax_index_rolling_mean_7,Vax_index_rolling_std_7,Vax_index_rolling_mean_14,Vax_index_rolling_std_14,new_confirmed_rolling_mean_7,new_confirmed_rolling_std_7,new_confirmed_rolling_mean_14,new_confirmed_rolling_std_14
0,2020-04-01,0.0,23332,12059.0,3099,3989.0,694.0,35571.0,4730.0,56171302,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-04-02,1494.0,26264,12135.0,2932,3895.0,725.0,39466.0,5455.0,56171302,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-04-03,1788.0,28828,13635.0,2564,3878.0,737.0,43344.0,6192.0,56171302,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-04-04,1950.0,31421,15469.0,2593,3260.0,828.0,46604.0,7020.0,56171302,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-04-05,2097.0,34013,16657.0,2592,2994.0,823.0,49598.0,7843.0,56171302,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
890,2022-09-08,127.0,848252,4881.0,508,3192.0,36.0,19692527.0,164139.0,56171302,...,520.928571,56.441207,0.06738,0.0,0.073008,0.006743,3492.142857,481.273113,3475.214286,571.305277
891,2022-09-09,122.0,848674,4797.0,422,2987.0,36.0,19695514.0,164175.0,56171302,...,514.500000,62.352287,0.06738,0.0,0.072070,0.006529,3464.571429,506.977599,3479.857143,566.709240
892,2022-09-10,129.0,849082,4650.0,408,2756.0,24.0,19698270.0,164199.0,56171302,...,509.714286,67.938240,0.06738,0.0,0.071132,0.006156,3439.714286,540.700075,3494.571429,542.882296
893,2022-09-11,132.0,849581,4692.0,499,3768.0,27.0,19702038.0,164226.0,56171302,...,506.857143,67.451244,0.06738,0.0,0.070194,0.005591,3508.714286,548.429185,3560.785714,512.730425


In [ ]:
window_size_7 = 7
window_size_14 = 14

# List of columns for which rolling statistics need to be computed
columns_to_compute = ['covidOccupiedMVBeds', 'hospitalCases', 'newAdmissions', 'Vax_index', 'new_confirmed']

# Compute rolling statistics for each column
for column in columns_to_compute:
    data[f'{column}_rolling_mean_7'] = data[column].rolling(window=window_size_7).mean()
    # data[f'{column}_rolling_std_7'] = data[column].rolling(window=window_size_7).std()
    # data[f'{column}_rolling_mean_14'] = data[column].rolling(window=window_size_14).mean()
    # data[f'{column}_rolling_std_14'] = data[column].rolling(window=window_size_14).std()
    
data.head()

,date,covidOccupiedMVBeds,cumAdmissions,hospitalCases,newAdmissions,new_confirmed,new_deceased,cumulative_confirmed,cumulative_deceased,population,...,newAdmissions_rolling_mean_14,newAdmissions_rolling_std_14,Vax_index_rolling_mean_7,Vax_index_rolling_std_7,Vax_index_rolling_mean_14,Vax_index_rolling_std_14,new_confirmed_rolling_mean_7,new_confirmed_rolling_std_7,new_confirmed_rolling_mean_14,new_confirmed_rolling_std_14
0,2020-04-01,0.0,23332,12059.0,3099,3989.0,694.0,35571.0,4730.0,56171302,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-04-02,1494.0,26264,12135.0,2932,3895.0,725.0,39466.0,5455.0,56171302,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-04-03,1788.0,28828,13635.0,2564,3878.0,737.0,43344.0,6192.0,56171302,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-04-04,1950.0,31421,15469.0,2593,3260.0,828.0,46604.0,7020.0,56171302,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-04-05,2097.0,34013,16657.0,2592,2994.0,823.0,49598.0,7843.0,56171302,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
from datetime import datetime, timedelta
import holidays

def engineer_features(df):
    """
    Engineer additional features for ventilator demand forecasting.
    
    Args:
        df (pandas.DataFrame): The input DataFrame containing COVID-19 data
        
    Returns:
        pandas.DataFrame: DataFrame with additional engineered features
    """
    # Create a copy to avoid modifying the original DataFrame
    data = df.copy()
    
    # Ensure date is in datetime format
    if isinstance(data['date'].iloc[0], str):
        data['date'] = pd.to_datetime(data['date'])
    
    # Sort by date to ensure proper calculation of time-based features
    data = data.sort_values('date').reset_index(drop=True)
    
    # 1. Rate of change (daily) for hospital cases
    data['hospitalCases_daily_change'] = data['hospitalCases'].diff()
    data['hospitalCases_pct_change'] = data['hospitalCases'].pct_change() * 100
    
    # 2. Rate of change (daily) for new admissions
    data['newAdmissions_daily_change'] = data['newAdmissions'].diff()
    data['newAdmissions_pct_change'] = data['newAdmissions'].pct_change() * 100
    
    # 3. Percentage of hospital cases requiring ventilation
    data['pct_cases_ventilated'] = (data['covidOccupiedMVBeds'] / data['hospitalCases']) * 100
    
    # 4. Rate of change in ventilator usage
    data['vent_daily_change'] = data['covidOccupiedMVBeds'].diff()
    data['vent_pct_change'] = data['covidOccupiedMVBeds'].pct_change() * 100
    
    # 5. Three-day momentum features (change over 3 days)
    data['hospitalCases_3day_momentum'] = data['hospitalCases'].diff(3)
    data['newAdmissions_3day_momentum'] = data['newAdmissions'].diff(3)
    data['vent_3day_momentum'] = data['covidOccupiedMVBeds'].diff(3)
    
    # 6. Ratio features
    data['admission_to_hospital_ratio'] = data['newAdmissions'] / data['hospitalCases']
    
    # 7. Weekend flag (0 for weekday, 1 for weekend)
    # Assuming day_of_week is 0-based with 0,6 being weekend (can adjust if needed)
    data['is_weekend'] = data['day_of_week'].apply(lambda x: 1 if x in [5, 6] else 0)
    
    # 8. UK holiday flags
    uk_holidays = holidays.UK()
    data['is_holiday'] = data['date'].apply(lambda x: 1 if x in uk_holidays else 0)
    
    # 9. Days since most recent peak in ventilator usage
    def days_since_peak(series):
        result = np.zeros(len(series))
        current_max = series.iloc[0]
        current_max_idx = 0
        
        for i in range(1, len(series)):
            if series.iloc[i] > current_max:
                current_max = series.iloc[i]
                current_max_idx = i
            
            result[i] = i - current_max_idx
        
        return result
    
    data['days_since_vent_peak'] = days_since_peak(data['covidOccupiedMVBeds'])
    
    # 10. Acceleration features (change in the rate of change)
    data['hospitalCases_acceleration'] = data['hospitalCases_daily_change'].diff()
    data['vent_acceleration'] = data['vent_daily_change'].diff()
    
    # 11. Moving average ratios (Short-term vs long-term trends)
    data['hospital_trend_ratio'] = data['hospitalCases_rolling_mean_7'] / data['hospitalCases_rolling_mean_14']
    data['vent_trend_ratio'] = data['covidOccupiedMVBeds_rolling_mean_7'] / data['covidOccupiedMVBeds_rolling_mean_14']
    
    # 12. Lag features for ventilator demand (t-1, t-7, t-14)
    data['vent_lag_1'] = data['covidOccupiedMVBeds'].shift(1)
    data['vent_lag_7'] = data['covidOccupiedMVBeds'].shift(7)
    data['vent_lag_14'] = data['covidOccupiedMVBeds'].shift(14)
    
    # 13. Lag features for hospital cases
    data['hospital_lag_1'] = data['hospitalCases'].shift(1)
    data['hospital_lag_7'] = data['hospitalCases'].shift(7)
    
    # 14. Season indicators (simplified)
    # Winter (Dec-Feb), Spring (Mar-May), Summer (Jun-Aug), Fall (Sep-Nov)
    data['season'] = data['month'].apply(
        lambda x: 1 if x in [12, 1, 2] else 
                  2 if x in [3, 4, 5] else 
                  3 if x in [6, 7, 8] else 4
    )
    # Convert to one-hot encoding
    season_dummies = pd.get_dummies(data['season'], prefix='season')
    data = pd.concat([data, season_dummies], axis=1)
    
    # 15. COVID wave indicator (simplified approach)
    # This is a basic approach - a more sophisticated approach would use actual wave dates
    def identify_waves(series, threshold_multiplier=1.5):
        # Use rolling mean to smooth the data
        smooth = series.rolling(window=14, min_periods=1).mean()
        # Calculate the overall mean
        overall_mean = smooth.mean()
        # Mark as wave when above threshold
        return (smooth > (overall_mean * threshold_multiplier)).astype(int)
    
    data['covid_wave'] = identify_waves(data['covidOccupiedMVBeds'])
    
    # Fill NaN values created by diff and shift operations
    # For trend ratios and percentages, forward fill might be better
    ratio_cols = ['hospital_trend_ratio', 'vent_trend_ratio', 'pct_cases_ventilated', 
                  'admission_to_hospital_ratio']
    data[ratio_cols] = data[ratio_cols].fillna(method='ffill')
    
    # For other columns with NaNs, use 0 (assuming start of the series)
    data = data.fillna(0)
    
    return data

def select_features_for_modeling(df, target_col='covidOccupiedMVBeds', prediction_horizon=7):
    """
    Select relevant features for the forecasting model and prepare train/test split.
    
    Args:
        df (pandas.DataFrame): The DataFrame with all features
        target_col (str): The target column to predict
        prediction_horizon (int): Days ahead to predict
        
    Returns:
        tuple: (X_train, y_train, X_val, y_val, X_test, y_test, feature_names)
    """
    # Features that are likely useful for the model
    selected_features = [
        # Original key features
        'hospitalCases', 'newAdmissions', 'Vax_index',
        
        # Rolling statistics
        'hospitalCases_rolling_mean_7', 'hospitalCases_rolling_std_7',
        'newAdmissions_rolling_mean_7', 'newAdmissions_rolling_std_7',
        'new_confirmed_rolling_mean_7',
        
        # Engineered features
        'hospitalCases_daily_change', 'hospitalCases_pct_change',
        'pct_cases_ventilated', 'vent_daily_change', 'vent_pct_change',
        'hospitalCases_3day_momentum', 'newAdmissions_3day_momentum',
        'vent_3day_momentum', 'admission_to_hospital_ratio',
        'is_weekend', 'is_holiday', 'days_since_vent_peak',
        'hospitalCases_acceleration', 'vent_acceleration',
        'hospital_trend_ratio', 'vent_trend_ratio',
        
        # Lag features
        'vent_lag_1', 'vent_lag_7', 'vent_lag_14',
        'hospital_lag_1', 'hospital_lag_7',
        
        # Seasonal features
        'season_1', 'season_2', 'season_3', 'season_4',
        'covid_wave',
        
        # Time components
        'day_of_week'
    ]
    
    # Create future target (what we want to predict)
    df['future_target'] = df[target_col].shift(-prediction_horizon)
    
    # Remove rows with NaN in the future target (at the end of the dataset)
    modeling_df = df.dropna(subset=['future_target']).copy()
    
    # Split into features and target
    X = modeling_df[selected_features]
    y = modeling_df['future_target']
    
    # Split into train, validation, and test sets
    # Using time-based split for time series data
    train_size = int(len(X) * 0.7)
    val_size = int(len(X) * 0.15)
    
    X_train = X.iloc[:train_size]
    y_train = y.iloc[:train_size]
    
    X_val = X.iloc[train_size:train_size+val_size]
    y_val = y.iloc[train_size:train_size+val_size]
    
    X_test = X.iloc[train_size+val_size:]
    y_test = y.iloc[train_size+val_size:]
    
    return X_train, y_train, X_val, y_val, X_test, y_test, selected_features

In [20]:

# Find the minimum and maximum dates
min_date = data['date'].min()
max_date = data['date'].max()

print("Minimum Date:", min_date)
print("Maximum Date:", max_date)

# Calculate the date ranges for train, val, and test sets
date_range = max_date - min_date
train_end = min_date + pd.Timedelta(days=date_range.days * 0.75)
val_end = train_end + pd.Timedelta(days=date_range.days * 0.10)

# Split the data into train, validation, and test sets based on the date ranges
train = data[data['date'] < train_end]
val = data[(data['date'] >= train_end) & (data['date'] < val_end)]
test = data[data['date'] >= val_end]

# Calculate the percentage of dates in each dataset
total_samples = len(data)
train_percentage = len(train) / total_samples * 100
val_percentage = len(val) / total_samples * 100
test_percentage = len(test) / total_samples * 100

print(f"# of Training samples: {len(train)} | # of Validation samples: {len(val)} | # of Test samples: {len(test)}")
print(f"Percentage of Dates in Train: {train_percentage:.2f}% | Percentage of Dates in Validation: {val_percentage:.2f}% | Percentage of Dates in Test: {test_percentage:.2f}%")
print(f"Max Date in Train: {train.date.max()} | Min Date in Validation: {val.date.min()} | Min Date in Test: {test.date.min()}")


Minimum Date: 2020-04-01 00:00:00
Maximum Date: 2022-09-12 00:00:00
# of Training samples: 671 | # of Validation samples: 89 | # of Test samples: 135
Percentage of Dates in Train: 74.97% | Percentage of Dates in Validation: 9.94% | Percentage of Dates in Test: 15.08%
Max Date in Train: 2022-01-31 00:00:00 | Min Date in Validation: 2022-02-01 00:00:00 | Min Date in Test: 2022-05-01 00:00:00


In [21]:
# drop the 'areaName' column as it's no longer needed
train = train.drop('areaName', axis=1)
val = val.drop('areaName', axis=1)
test = test.drop('areaName', axis=1)


In [22]:
# Convert date to datetime and set as index
train.set_index("date", inplace=True)
test.set_index("date", inplace=True)
val.set_index("date", inplace=True)

In [23]:
# Concatenate the DataFrames
sample_df = pd.concat([train, val, test])

# Convert feature columns to float32
# Exclude the 'type' column from conversion as it's a string column
for col in sample_df.columns:
    if col != "type":
        sample_df[col] = sample_df[col].astype("float32")

sample_df.head()

,covidOccupiedMVBeds,cumAdmissions,hospitalCases,newAdmissions,new_confirmed,new_deceased,cumulative_confirmed,cumulative_deceased,population,openstreetmap_id,...,newAdmissions_rolling_mean_14,newAdmissions_rolling_std_14,Vax_index_rolling_mean_7,Vax_index_rolling_std_7,Vax_index_rolling_mean_14,Vax_index_rolling_std_14,new_confirmed_rolling_mean_7,new_confirmed_rolling_std_7,new_confirmed_rolling_mean_14,new_confirmed_rolling_std_14
date,,,,,,,,,,,,,,,,,,,,,
2020-04-01,0.0,23332.0,12059.0,3099.0,3989.0,694.0,35571.0,4730.0,56171304.0,151336.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-04-02,1494.0,26264.0,12135.0,2932.0,3895.0,725.0,39466.0,5455.0,56171304.0,151336.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-04-03,1788.0,28828.0,13635.0,2564.0,3878.0,737.0,43344.0,6192.0,56171304.0,151336.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-04-04,1950.0,31421.0,15469.0,2593.0,3260.0,828.0,46604.0,7020.0,56171304.0,151336.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-04-05,2097.0,34013.0,16657.0,2592.0,2994.0,823.0,49598.0,7843.0,56171304.0,151336.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
columns_to_select = [
    'covidOccupiedMVBeds_rolling_mean_7',
    // 'covidOccupiedMVBeds_rolling_mean_14',
    'hospitalCases',
    'newAdmissions',
    'new_confirmed',
    'year',
    'month',
    'day_of_week',
    'Vax_index'
]

In [14]:
sample_df = sample_df[columns_to_select]
sample_df.head()

,covidOccupiedMVBeds,hospitalCases,newAdmissions,new_confirmed,new_deceased,month,day_of_week,Vax_index
date,,,,,,,,
2020-04-01,0.0,12059.0,3099.0,3989.0,694.0,4.0,2.0,0.33
2020-04-02,1494.0,12135.0,2932.0,3895.0,725.0,4.0,3.0,0.33
2020-04-03,1788.0,13635.0,2564.0,3878.0,737.0,4.0,4.0,0.33
2020-04-04,1950.0,15469.0,2593.0,3260.0,828.0,4.0,5.0,0.33
2020-04-05,2097.0,16657.0,2592.0,2994.0,823.0,4.0,6.0,0.33


In [15]:
cols = list(sample_df.columns)
cols.remove("covidOccupiedMVBeds")
sample_df = sample_df[cols + ["covidOccupiedMVBeds"]]

In [16]:
target = "covidOccupiedMVBeds"
pred_df = pd.concat([train[[target]], val[[target]]])

In [17]:
sample_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 895 entries, 2020-04-01 to 2022-09-12
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   hospitalCases        895 non-null    float32
 1   newAdmissions        895 non-null    float32
 2   new_confirmed        895 non-null    float32
 3   new_deceased         895 non-null    float32
 4   month                895 non-null    float32
 5   day_of_week          895 non-null    float32
 6   Vax_index            895 non-null    float32
 7   covidOccupiedMVBeds  895 non-null    float32
dtypes: float32(8)
memory usage: 35.0 KB


In [18]:
datamodule = TimeSeriesDataModule(
    data=sample_df,
    n_val=val.shape[0],
    n_test=test.shape[0],
    window=7, 
    horizon=1,  
    normalize="global",  
    batch_size=32,
    num_workers=0,
)
datamodule.setup()

In [19]:
rnn_config = SingleStepRNNConfig(
    rnn_type="RNN",
    input_size=8,  
    hidden_size=64,
    num_layers=3,
    bidirectional=False,
    learning_rate=1e-3
)
model = SingleStepRNNModel(rnn_config)
model.float()

SingleStepRNNModel(
  (rnn): RNN(8, 64, num_layers=3, batch_first=True)
  (fc): Linear(in_features=64, out_features=1, bias=True)
  (loss): MSELoss()
)

In [20]:
import pytorch_lightning as pl
trainer = pl.Trainer(
    min_epochs=5,
    max_epochs=100,
    callbacks=[pl.callbacks.EarlyStopping(monitor="valid_loss", patience=5)],
)
trainer.fit(model, datamodule)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type    | Params
---------------------------------
0 | rnn  | RNN     | 21.4 K
1 | fc   | Linear  | 65    
2 | loss | MSELoss | 0     
---------------------------------
21.4 K    Trainable params
0         Non-trainable params
21.4 K    Total params
0.086     Total estimated model params size (MB)


c:\Users\olarinoyem\AppData\Local\miniconda3\envs\tf\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: PossibleUserWarning:

The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.

c:\Users\olarinoyem\AppData\Local\miniconda3\envs\tf\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: PossibleUserWarning:

The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.

c:\Users\olarinoyem\AppData\Local\miniconda3\envs\tf\lib\site-packages\pytorch_lightning\loops\fit_loop.py:293: PossibleUserWarning:

The number of training batches (21) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want 

Epoch 11: 100%|██████████| 21/21 [00:00<00:00, 82.64it/s, v_num=14, train_loss=0.000154, valid_loss=0.00477, valid_MAE=0.0658, train_MAE=0.0112] 


In [21]:
# Removing artifacts created during training
import shutil
shutil.rmtree("lightning_logs")

In [22]:
def mase(actual, predicted, insample_actual):
    mae_insample = np.mean(np.abs(np.diff(insample_actual)))
    mae_outsample = np.mean(np.abs(actual - predicted))
    return mae_outsample / mae_insample


def forecast_bias(actual, predicted):
    return np.mean(predicted - actual)


In [23]:
def plot_forecast(pred_df, forecast_columns, forecast_display_names=None, save_path=None):
    if forecast_display_names is None:
        forecast_display_names = forecast_columns
    else:
        assert len(forecast_columns) == len(forecast_display_names)

    mask = ~pred_df[forecast_columns[0]].isnull()
    colors = [
        "rgba(" + ",".join([str(c) for c in plotting_utils.hex_to_rgb(c)]) + ",<alpha>)"
        for c in px.colors.qualitative.Plotly
    ]
    act_color = colors[0]
    colors = cycle(colors[1:])
    
    fig = go.Figure()
    fig.add_trace(
        go.Scatter(
            x=pred_df[mask].index,
            y=pred_df[mask].covidOccupiedMVBeds_trend_diff,
            mode="lines",
            line=dict(color=act_color.replace("<alpha>", "0.9")),
            name="7 day-moving average MVBeds_trends",
        )
    )
    for col, display_col in zip(forecast_columns, forecast_display_names):
        fig.add_trace(
            go.Scatter(
                x=pred_df[mask].index,
                y=pred_df.loc[mask, col],
                mode="lines",
                line=dict(dash="dot", color=next(colors).replace("<alpha>", "1")),
                name=display_col,
            )
        )
    
    return fig


def highlight_abs_min(s, props=""):
    return np.where(s == np.nanmin(np.abs(s.values)), props, "")


In [24]:
metric_record = []

In [ ]:
from sklearn.metrics import (
    mean_absolute_error as mae,
    mean_squared_error as mse,
    mean_absolute_percentage_error as mape,
)
import torch

# Get predictions for test set
pred = trainer.predict(model, datamodule.test_dataloader())
pred = torch.cat(pred).squeeze().detach().numpy()

# Denormalize predictions using datamodule's statistics
pred = pred * datamodule.train.std + datamodule.train.mean

# Get actual values from test set
actuals = test[target].values

# Calculate metrics
metrics = {
    "Algorithm": rnn_config.rnn_type,
    "MAE": mae(actuals, pred),
    "MSE": mse(actuals, pred),
    "RMSE": np.sqrt(mse(actuals, pred)),
    "MAPE": mape(actuals, pred),
    "MASE": mase(actuals, pred, train[target].values),
    "Forecast Bias": forecast_bias(actuals, pred),
}

# Format metrics for display
value_formats = ["{}", "{:.4f}", "{:.4f}", "{:.4f}", "{:.4f}", "{:.4f}", "{:.2f}"]
metrics = {key: format_.format(value) for key, value, format_ in zip(metrics.keys(), metrics.values(), value_formats)}
metric_record.append(metrics)
print(metrics)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\olarinoyem\AppData\Local\miniconda3\envs\tf\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: PossibleUserWarning:

The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.



Predicting DataLoader 0: 100%|██████████| 5/5 [00:00<00:00, 237.09it/s]
{'Algorithm': 'RNN', 'MAE': '641.5645', 'MSE': '480659.6045', 'RMSE': '693.2962', 'MAPE': '3.3716', 'MASE': '24.1393', 'Forecast Bias': '641.56'}


In [32]:
# Create DataFrame with predictions
pred_df = pd.DataFrame({"Vanilla RNN": pred}, index=test.index)
pred_df = test[['covidOccupiedMVBeds']].join(pred_df)

# Plot the forecast
fig = plot_forecast(pred_df, forecast_columns=["Vanilla RNN"], forecast_display_names=["Vanilla RNN"])
title = f"{rnn_config.rnn_type}: MAE: {metrics['MAE']} | MSE: {metrics['MSE']} | MASE: {metrics['MASE']} | Bias: {metrics['Forecast Bias']}"
fig = format_plot(fig, title=title)
fig.update_xaxes(type="date", range=["2022-05-01", "2022-09-12"])
fig.show()